# 직업 코드 일치도 분석

이 노트북은 제공된 직업 분류표를 사용하여 직업 코드 간의 일치도를 분석하고, 
일치도에 따라 점수를 부여합니다. 
일치도 2부터 5까지의 분포도를 계산하고, MinMaxScaler를 사용하여 점수를 부여합니다.


In [1]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Excel file 불러오기
file_path = '직업분류표.xlsx'
xls = pd.ExcelFile(file_path)

# 특정 sheet을 DataFrame으로 불러오기
df = pd.read_excel(file_path, sheet_name='분류 항목표(영문포함, 목록형)')

# 전처리
df = df[['코드', '국문 항목명']]

# 직업 코드 관계 함수 생성
def calculate_match_degree(code1, code2):
    match_degree = 0
    for i in range(min(len(code1), len(code2))):
        if code1[i] == code2[i]:
            match_degree += 1
        else:
            break
    return match_degree

# 직업 일치도 계산
from itertools import combinations

match_degrees = []

codes = df['코드'].astype(str).tolist()

for code1, code2 in combinations(codes, 2):
    match_degree = calculate_match_degree(code1, code2)
    if match_degree > 1:  # Exclude match degree 0 and 1
        match_degrees.append((code1, code2, match_degree))

# 직업일치도에 따른 DataFrame 생성
match_df = pd.DataFrame(match_degrees, columns=['코드1', '코드2', '일치도'])

# 직업일치도 DataFrame 전처리
filtered_match_df = match_df[match_df['일치도'].between(2, 5)]

# 0~1 MinMaxScaler로 가중치 계산
scaler = MinMaxScaler(feature_range=(0, 1))
filtered_match_df['점수'] = scaler.fit_transform(filtered_match_df[['일치도']])

# 각 코드가 최소한 하나의 동일한 코드를 갖고 있다고 가정하고 점수를 조정 (일치도 5)
unique_codes = df['코드'].astype(str).unique()
identical_codes = [(code, code, 5) for code in unique_codes]
identical_df = pd.DataFrame(identical_codes, columns=['코드1', '코드2', '일치도'])
adjusted_match_df = pd.concat([filtered_match_df, identical_df])

# MinMaxScaler를 사용하여 일치도를 다시 정규화
adjusted_match_df['점수'] = scaler.fit_transform(adjusted_match_df[['일치도']])

# 일치도 2에서 5에 대한 총 개수를 계산
adjusted_distribution = adjusted_match_df['일치도'].value_counts().sort_index()
adjusted_distribution[1] = 0
total_count = adjusted_distribution.sum()

# 적절한 공식을 사용하여 조정된 점수를 다시 계산
adjusted_scores_corrected = 1 - (adjusted_distribution / total_count)
adjusted_scores_corrected[1] = 0
adjusted_scores_corrected[5] = 1

# DataFrame에서 일치도에 조정된 점수를 매핑
adjusted_match_df['adjusted_점수_corrected'] = adjusted_match_df['일치도'].map(adjusted_scores_corrected)

# '일치도'와 'adjusted_점수_corrected' 열을 추출하여 새 DataFrame을 생성
new_corrected_df = adjusted_match_df[['일치도', 'adjusted_점수_corrected']].drop_duplicates().reset_index(drop=True)
new_corrected_df.rename(columns={'adjusted_점수_corrected': '점수'}, inplace=True)
new_corrected_df['점수'] = new_corrected_df['점수'].round(2)
new_corrected_df = new_corrected_df.sort_values(by='일치도').reset_index(drop=True)

# Display the final DataFrame
new_corrected_df


,일치도,점수
0,2,0.28
1,3,0.83
2,4,0.93
3,5,1.00
